In [ ]:
!pip install boto3

In [1]:
import utils as u
import os

In [2]:
if not 'workbookDir' in globals():
    workbookDir = os.getcwd()

In [3]:
aws_separator = "/"

In [4]:
# Import S3 connectors librairies
#from awsPy.aws_authorization import aws_connector
import importlib.util
spec = importlib.util.spec_from_file_location("aws_connector.py", "C:/Users/Hp/Documents/GitHub/aws-python/awsPy/aws_authorization/aws_connector.py")
aws_connector = importlib.util.module_from_spec(spec)
spec.loader.exec_module(aws_connector)

#from awsPy.aws_s3 import service_s3
import importlib.util
spec = importlib.util.spec_from_file_location("service_s3.py", "C:/Users/Hp/Documents/GitHub/aws-python/awsPy/aws_s3/service_s3.py")
service_s3 = importlib.util.module_from_spec(spec)
spec.loader.exec_module(service_s3)

In [5]:
# Connect to S3
import os
from pathlib import Path

bucket = 'calfdata'
path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = "{}/programme_matching/credential_AWS.json".format(parent_path)

con = aws_connector.aws_instantiate(credential = path_cred,
                                        region = 'eu-west-3')
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = 'calfdata')

In [7]:
# List files to send and compute each destination
prep_path='data/prep_flux_2017_horsETS' #flux 2017 hors ETS

list_files=[]
for root,dirs,files in os.walk(prep_path):
    for file in files:
          if file.endswith(('.csv')):
                #print(file)
                list_files.append((file, u.calc_dest(filename=file)))
list_files[:5]

[('0101_100_20171003_074607_11_obs.csv',
  'INPI/TC_1/01_donnee_source/Flux/2017/OBS/NEW'),
 ('0101_100_20171003_074607_12_actes.csv',
  'INPI/TC_1/01_donnee_source/Flux/2017/ACTES/NEW'),
 ('0101_100_20171003_074607_13_comptes_annuels.csv',
  'INPI/TC_1/01_donnee_source/Flux/2017/COMPTES_ANNUELS/NEW'),
 ('0101_100_20171003_074607_1_PM.csv',
  'INPI/TC_1/01_donnee_source/Flux/2017/PM/NEW'),
 ('0101_100_20171003_074607_2_PM_EVT.csv',
  'INPI/TC_1/01_donnee_source/Flux/2017/PM/EVT')]

In [8]:
len(list_files)

236810

In [10]:
# Send files to s3 (prepared 2017)
for (file_to_upload,destination_in_s3) in list_files:
    # if file exist in S3?
    full_path = os.path.join(prep_path, file_to_upload)
    #s3.upload_file(full_path, destination_in_s3)
   
    key = aws_separator.join([destination_in_s3,file_to_upload])
    if not s3.key_exist(key):
        s3.upload_file(full_path, destination_in_s3)